#Lab1
##0-1Knapsack

In [100]:
import numpy as np
from icecream import ic

###Thanks to professor

In [101]:
NUM_KNAPSACKS = 3
NUM_ITEMS = 10
NUM_DIMENSIONS = 2

In [102]:
VALUES = np.random.randint(0, 100, size=NUM_ITEMS)
WEIGHTS = np.random.randint(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = np.random.randint(0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS,NUM_DIMENSIONS))

In [103]:
print(VALUES)
print(WEIGHTS)
CONSTRAINTS

[61 34 14 90  7 87 74 42 75 98]
[[28 57]
 [18 27]
 [34 52]
 [97 97]
 [ 5 91]
 [15 28]
 [38 77]
 [71 22]
 [27 33]
 [36 85]]


array([[134,  74],
       [242,  97],
       [ 72,  24]])

###TEST PROBLEMS

In [104]:
#test some solutions
#start from classic knapsack solution
#random solution
def random_feasible_init(values, weights, constraints, num_knapsacks, rng=None):
    rng = np.random.default_rng() if rng is None else rng
    N, D = weights.shape
    K = num_knapsacks
    sol = np.zeros((N, K), dtype=int)
    loads = np.zeros((K, D), dtype=int)
    for i in rng.permutation(N):
        for k in rng.permutation(K):
            if np.all(loads[k] + weights[i] <= constraints):
                sol[i, k] = 1
                loads[k] += weights[i]
                break
    return sol


In [105]:
def fitness(sol, values):
    # only return the fitness value
    return int((sol * values[:, None]).sum())

def knapsack_loads(sol, weights):
    return sol.T @ weights  # (K, D)

In [106]:
#local search
#neighbor
def best_improving_neighbor(sol, values, weights, constraints):
    N, K = sol.shape
    cur_fit = fitness(sol, values)
    best_delta = 0
    best_move = None
    loads = knapsack_loads(sol, weights)

    for i in range(N):
        if sol[i].any():  # 已放入就跳过（此简单版只尝试 add）
            continue
        for k_to in range(K):
            if np.all(loads[k_to] + weights[i] <= constraints):
                delta = values[i]
                if delta > best_delta:
                    best_delta = delta
                    best_move = ("add", i, None, k_to)

    if best_move is None:
        return None, 0, None

    op, i, k_from, k_to = best_move
    new_sol = sol.copy()
    new_sol[i, :] = 0
    new_sol[i, k_to] = 1
    new_fit = fitness(new_sol, values)
    return new_sol, (new_fit - cur_fit), {"op": op, "i": i, "k_from": k_from, "k_to": k_to}


In [ ]:
#local search
#hill climbing


def hill_climb(values, weights, constraints, num_knapsacks, *, 
               restarts=10, max_iters=10000, seed=42,
               debug=True, log_every=1):
    """
    debug=True 开启 icecream 输出；
    log_every 控制节流：每接受 log_every 次改进打印一次（1=每次都打）
    仅在本函数内用 ic 打印过程
    """
    rng = np.random.default_rng(seed)
    best_sol, best_fit = None, -1

    for r in range(restarts):
        sol = random_feasible_init(values, weights, constraints, num_knapsacks, rng)
        cur_fit = fitness(sol, values)
        if debug:
            ic(f"[restart {r+1}/{restarts}] init_fit={int(cur_fit)}",
               "loads", knapsack_loads(sol, weights))

        it, accepted = 0, 0
        while it < max_iters:
            loads_before = knapsack_loads(sol, weights)
            fit_before = cur_fit

            neighbor, gain, info = best_improving_neighbor(sol, values, weights, constraints)

            if neighbor is None or gain <= 0:
                if debug:
                    ic(f"[restart {r+1}] reached local optimum at it={it}, fit={int(fit_before)}")
                break

            # 接受邻居
            sol = neighbor
            cur_fit = fitness(sol, values)
            loads_after = knapsack_loads(sol, weights)
            dloads = loads_after - loads_before

            accepted += 1
            it += 1

            if debug and (accepted % log_every == 0):
                # info: {op, i, k_from, k_to}
                ic(
                    f"[restart {r+1}] step={accepted}, "
                    f"op={info.get('op')}, item={info.get('i')}, "
                    f"from_k={info.get('k_from')}, to_k={info.get('k_to')}, "
                    f"gain=+{int(gain)}, fit={int(cur_fit)}",
                    "Δloads", dloads,
                    "loads", loads_after
                )

        fit = cur_fit
        if debug:
            ic(f"[restart {r+1}] local best fit={int(fit)}", "loads", knapsack_loads(sol, weights))

        if fit > best_fit:
            best_fit = fit
            best_sol = sol

    return best_sol, int(best_fit)


In [108]:
# Problem 1:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=NUM_DIMENSIONS)

# sol = greedy_solution(VALUES, WEIGHTS, CONSTRAINTS, NUM_KNAPSACKS)
# print(sol)



# che = check_solution(sol, WEIGHTS, CONSTRAINTS)
# print(che)
# #try fitness function


# sol0 = random_feasible_init(VALUES, WEIGHTS, CONSTRAINTS, NUM_KNAPSACKS)
# print(sol0)

sol, fit = hill_climb(
        VALUES, WEIGHTS, CONSTRAINTS, NUM_KNAPSACKS,
        restarts=20, max_iters=5000, seed=42,
        debug=True,     # ← 开启 icecream
        log_every=1     # ← 每次改进都打印；可改为 5/10 节流
    )


# 本地爬山优化
# sol_best, best_val = local_search(VALUES, WEIGHTS, CONSTRAINTS, sol0)
# print("Best value:", best_val)


ic| f"[restart {r+1}/{restarts}] init_fit={int(cur_fit)}": '[restart 1/20] init_fit=1065'
    "loads": 'loads'
    knapsack_loads(sol, weights): array([[287, 316],
                                         [355, 443],
                                         [343, 313]])
ic| f"[restart {r+1}] reached local optimum at it={it}, fit={int(fit_before)}": '[restart 1] reached local optimum at it=0, fit=1065'
ic| f"[restart {r+1}] local best fit={int(fit)}": '[restart 1] local best fit=1065'
    "loads": 'loads'
    knapsack_loads(sol, weights): array([[287, 316],
                                         [355, 443],
                                         [343, 313]])
ic| f"[restart {r+1}/{restarts}] init_fit={int(cur_fit)}": '[restart 2/20] init_fit=1065'
    "loads": 'loads'
    knapsack_loads(sol, weights): array([[407, 494],
                                         [365, 308],
                                         [213, 270]])
ic| f"[restart {r+1}] reached local optimum at it={it}, fit

In [109]:
# Problem 2:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(1000 * 2, 1000 * NUM_ITEMS // NUM_KNAPSACKS, size=NUM_DIMENSIONS)

greedy_solution = np.zeros((NUM_ITEMS, NUM_KNAPSACKS), dtype=int)
item_indices = np.argsort(-VALUES / np.sum(WEIGHTS, axis=1))    
knapsack_weights = np.zeros((NUM_KNAPSACKS, NUM_DIMENSIONS), dtype=int)
for idx in item_indices:
    for k in range(NUM_KNAPSACKS):
        if np.all(knapsack_weights[k] + WEIGHTS[idx] <= CONSTRAINTS):
            greedy_solution[idx, k] = 1
            knapsack_weights[k] += WEIGHTS[idx]
            break
print(greedy_solution)

#check feasibility
def is_feasible(solution):
    knapsack_weights = np.zeros((NUM_KNAPSACKS, NUM_DIMENSIONS), dtype=int)
    for i in range(NUM_ITEMS):
        for k in range(NUM_KNAPSACKS):
            if solution[i, k] == 1:
                knapsack_weights[k] += WEIGHTS[i]
    return np.all(knapsack_weights <= CONSTRAINTS)
is_feasible(greedy_solution)

[[0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0]
 [0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 1 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 1

np.True_

In [110]:
# Problem 3:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 100
NUM_ITEMS = 5000
NUM_DIMENSIONS = 100
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(1000 * 10, 1000 * 2 * NUM_ITEMS // NUM_KNAPSACKS, size=NUM_DIMENSIONS)


In [111]:
print(VALUES)
print(WEIGHTS)
print(CONSTRAINTS)

[ 89 773 654 ... 819 678 498]
[[197  33 627 ... 319 937 293]
 [550 543 242 ... 942 568 950]
 [436 411 836 ... 664 169 846]
 ...
 [869 582 968 ... 567 291 288]
 [ 97 220 662 ... 171 310 833]
 [656  51 146 ... 286 725 372]]
[79834 52005 17527 30007 59295 98921 12881 36174 63958 90566 25740 12251
 27746 78136 90704 39249 23072 69010 17326 29277 42886 48016 74931 93479
 30898 18137 75557 56030 75506 90246 42093 34668 22075 79709 98971 15032
 57157 21369 76865 11589 82666 78074 68715 85558 61242 80943 26905 84378
 85069 77221 16434 28156 17827 83548 72921 27512 41395 23936 72519 96465
 12904 72709 81023 81015 28959 98041 90482 89802 76408 29899 51171 94767
 78351 57860 58504 40545 97025 99185 42888 54965 24945 51281 31276 61913
 96714 93034 72901 95222 37857 53461 15463 71355 64979 70086 48286 38026
 60109 78500 56643 69915]


In [112]:
rng = np.random.default_rng(0)
NUM_KNAPSACKS = 2
NUM_ITEMS = 3
NUM_DIMENSIONS = 2

VALUES   = rng.integers(0, 10, size=NUM_ITEMS)              # (3,)
WEIGHTS  = rng.integers(0, 10, size=(NUM_ITEMS, NUM_DIMENSIONS))  # (3,2)
# 单背包/共享容量（不推荐多背包用）
CONSTRAINTS_shared = rng.integers(5, 15, size=NUM_DIMENSIONS)      # (2,)
# 多背包容量（推荐多背包用）
CONSTRAINTS_multi  = rng.integers(5, 15, size=(NUM_KNAPSACKS, NUM_DIMENSIONS))  # (2,2)

print("VALUES.shape:", VALUES.shape)            # (3,)
print("WEIGHTS.shape:", WEIGHTS.shape)          # (3,2)
print("共享容量:", CONSTRAINTS_shared.shape)     # (2,)
print("多背包容量:", CONSTRAINTS_multi.shape)    # (2,2)

print("VALUES =", VALUES)
print("WEIGHTS=\n", WEIGHTS)
print("CONSTRAINTS_multi=\n", CONSTRAINTS_multi)


VALUES.shape: (3,)
WEIGHTS.shape: (3, 2)
共享容量: (2,)
多背包容量: (2, 2)
VALUES = [8 6 5]
WEIGHTS=
 [[2 3]
 [0 0]
 [0 1]]
CONSTRAINTS_multi=
 [[14 10]
 [11 14]]
